In [20]:
from pathlib import Path
import geopandas as gpd
import folium

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas
# from IPython.display import display



In [44]:
path_results = Path(r'P:\moonshot2-casestudy\RA2CE_HACKATHON_JULY\results')
event = "TC_0631"
category = "clinic"

path_dir = path_results.joinpath(event)
assert path_dir.exists

path_no_hazard_OD_result = path_dir.joinpath("origin_closest_destination_optimal_routes_without_hazard.gpkg")
path_with_hazard_OD_result = path_dir.joinpath("origin_closest_destination_optimal_routes_with_hazard.gpkg")
path_origins = path_dir.joinpath("origin_closest_destination_origins.gpkg")
path_destinations = path_dir.joinpath("origin_closest_destination_destinations.gpkg")
assert path_no_hazard_OD_result.exists()

In [45]:
# Open the gdf
optimal_routes_no_hazard_gdf = gpd.read_file(path_no_hazard_OD_result)
optimal_routes_with_hazard_gdf = gpd.read_file(path_with_hazard_OD_result)
origins_gdf = gpd.read_file(path_origins)
destinations_gdf = gpd.read_file(path_destinations)



# Get the all the unique origins:
origins = optimal_routes_no_hazard_gdf["origin"].unique()


# keep only ctaegory hospital
destinations_gdf = destinations_gdf[destinations_gdf["category"] == category]
optimal_routes_with_hazard_gdf = optimal_routes_with_hazard_gdf[optimal_routes_with_hazard_gdf["category"] == category]
optimal_routes_no_hazard_gdf = optimal_routes_no_hazard_gdf[optimal_routes_no_hazard_gdf["category"] == category]


In [46]:
# clean destination gdf to account for combined destinations "B_5,B_6"
temp_gdf = gpd.GeoDataFrame()
for dest in destinations:
    if "," in dest:
        dests = dest.split(",")
        temp_gdf = temp_gdf.append(destinations_gdf[destinations_gdf["d_id"] == dests[0]])
        # rename the destination id to the combined destination id
        temp_gdf.loc[temp_gdf["d_id"] == dests[0], "d_id"] = dest
        
    else:
        temp_gdf = temp_gdf.append(destinations_gdf[destinations_gdf["d_id"] == dest])

destinations_gdf = temp_gdf

In [63]:
from numpy import array
lengths_no_hazard = []
lengths_with_hazard = []
differences = []
for origin in origins:
    
    optimal_routes_per_destination_no_hazard_gdf = optimal_routes_no_hazard_gdf[optimal_routes_no_hazard_gdf['origin'] == origin] # filter on destination B4
    optimal_routes_per_destination_with_hazard_gdf = optimal_routes_with_hazard_gdf[optimal_routes_with_hazard_gdf['origin'] == origin] # filter on destination B4
    
    
    lengths_no_hazard.append(optimal_routes_per_destination_no_hazard_gdf["lengthNorm"].max())
    lengths_with_hazard.append(optimal_routes_per_destination_with_hazard_gdf["lengthDisr"].max())
    
    if origin == "B_121":
        print(lengths_no_hazard[-1])
        print(lengths_with_hazard[-1])

differences = (array(lengths_with_hazard) - array(lengths_no_hazard)) / 1000  # conversion in km
    
    
print(lengths_no_hazard)    
print(lengths_with_hazard)
print(differences)
# origins["length_no_hazard"] = lengths_no_hazard
# origins["length_with_hazard"] = lengths_with_hazard
# origins["difference"] = differences


    


[10527.857, 35097.0, 24654.571000000004, 3583.0, 8294.0, 1503.0, 14465.0, 35374.812, 142843.41899999997, 52898.69699999999, 12856.386, 17677.0, 10297.0, 1535.0, 16364.0, 12238.0, 3717.0, 205.0, 12143.0, 9368.0, 3519.0, 16684.0, 1305.0, 4146.902, 156.0, 4300.0, 5266.0, 5284.0, 5209.0, 8426.0, 660.0, 24470.0, 58429.996000000036, 25967.54, 33533.0, 37615.259, 22357.0, 1955.0, 164934.949, 87960.41900000001, 9516.204, 59963.33399999999, 63061.6, 42219.0, 9932.0, 1121.0, 4401.9439999999995, 1083.0, 1558.0, 5920.0, 1695.0, 14450.517, 8737.0, 27785.0, 34692.0, 56437.0, 16390.175000000003, 15458.0, 26737.0, 173132.4420000001, 11173.0, 10444.399000000001, 11259.0, 12880.0, 152.0, 15736.0, 2351.0, 18850.0, 8640.508999999998, 8822.0, 10960.0, 6815.0, 13373.0, 38234.0, 7094.0, 18184.0, 29368.0, 19776.0, 2875.0, 58055.0, 28413.049999999992, 13506.0, 180053.7790000001, 17474.981000000003, 210797.15400000004, 19438.0, 35271.0, 21081.786, 18580.0, 641.0, 7434.0, 6816.6, 3097.0, 56429.0, 6116.0, 55866.6